# connectivity-based searchlight hyperalignment_tutorial     
### Erica Busch, 7/20
This is an example of how to run searchlight hyperalignment on a dataset. The example dataset here is the Grand   Budapest Hotel dataset, which has 5 runs for each of 21 subjects. This data has already been preprocessed and aligned to the fsaverage surface.   

In this example, we will compute the sparse connectivity matrices based on the first 4 runs of data. We then apply searchlight hyperalignment with a 20mm searchlight radius to these matrices to derive the common model. 



In [ ]:
import os,time,glob
import numpy as np
from scipy.stats import zscore
from mvpa2.datasets.base import Dataset
from mvpa2.misc.surfing.queryengine import SurfaceQueryEngine
from mvpa2.mappers.fxy import FxyMapper
from mvpa2.algorithms.searchlight_hyperalignment import SearchlightHyperalignment
from mvpa2.base import debug
from mvpa2.base.hdf5 import h5save, h5load
import dataset_utils as utils

In [ ]:
outdir = '/dartfs/rc/lab/D/DBIC/DBIC/f002d44/budapest/transformations/'

## 1. Load in PyMVPA datasets for each participant's data. 

In [ ]:
dss = utils.get_data()

## 2. Define what nodes are included in each searchlight using Dijkstra's distance metric
In general, I create these files once and save them to be reloaded each time I need them, which is often.  
This function creates a list of lists where the 0th element of each nested list is the index of the node around which the searchlight is built. The remaining elements in that list are nodes that fall within a searchlight on the given surface.

For more information, look up the documentation on PyMVPA SurfaceQueryEngine.


In [ ]:
# this function takes as input one subject's dataset (as a PyMVPA dataset), 
# which we're just using as a template, the target hemisphere, the searchlight radius, 
# and where you want to save this file. 
def compute_searchlights(ds, hemi, radius, outdir):
    from mvpa2.misc.surfing.queryengine import SurfaceQueryEngine
    # get the data for jsut the first participant
    node_indices = utils.get_node_indices(hemi)
    surf = get_freesurfer_surfaces(hemi)
    ds.fa['node_indices'] = node_indices.copy()
    qe = SurfaceQueryEngine(surf, radius)
    qe.train(ds)

    searchlights = []
    for idx in node_indices:
        sl = qe.query_byid(idx)
        searchlights.append(sl)
    savename = os.path.join(outdir,'{R}mm_searchlights_{S}h.npy'.format(R=radius,S=hemi))
    np.save(savename, searchlights)
    print('saved at: '+str(savename))
    return searchlights

In [ ]:
# this function just loads and returns your searchlights.
# since we did this on a hemisphere by hemisphere basis, we have to adjust if we're going to 
# use 
def load_searchlights(radius,hemi):
    if hemi == 'b':
        lh = load_searchlights(radius,'l')
        rh = load_searchlights(radius, 'r')
        adjusted_sls_rh = []
        for sl in rh:
            adjusted_sls_rh.append([b + len(lh) for b in sl])
        return np.concatenate((lh,adjusted_sls_rh),axis=0)
    return np.load(utils.basedir+'/{R}mm_searchlights_{S}h.npy'.format(R=radius,S=hemi),allow_pickle=True)

## 3. Define our target and seed node indices
To create connectivity targets, we will use 20mm searchlights defined on a sparse cortical surface (ico3, 642 nodes/hemisphere). Our connectivity seeds correspond to every vertex on the cortical surface of our data (in this case, ico5, 10242 nodes/hemisphere)



In [ ]:
# these are the node indices that will be used for creating connectivity targets
# defined on the sparse cortical surface after masking out the medial wall. This returns two lists:
# target_node_idx[0] is a list of the 588 node indices that are included in the left hemisphere
# and target_node_idx[1] are the 587 node indices in right hemisphere. 
sparse_resolution = 642
target_node_idx = utils.get_node_indices(hemi='b', surface_res=sparse_resolution)


# now we define all of the 'seed' node indices, which are all the nodes on the surface of our data
# after masking the medial wall. 
# this gives us two lists: seed_node_idx[0] are the 9372 nodes lh 
# seed_node_idx[1] are the 9370 rh
dense_resolution = 10242
seed_node_idx = utils.get_node_indices(hemi='b', surface_res=dense_resolution) 




## 4. Define our connectivity target function
We will average each time point across all nodes in the searchlight to get an average time series response for the searchlight. 
This 'mean feature measure' for each searchlight will serve as the connectivity targets.


In [ ]:
# pass this function a single subject's dataset (for that hemisphere, will be defined in thee function that calls this function) 
# and the list of searchlights for that hemisphere. 
# returns a numpy array where each item represents the average timecourse
# of all the nodes in that searchlight.
def compute_mean_features(ds, searchlights):
	mean_features = []
	for sl in searchlights:
		m = np.mean(ds[:,sl],axis=1)
		mean_features.append(zscore(m))
	return np.array(mean_features)

## 5. Define the function that builds our connectivity matrices.
This function takes as input all of your datasets (loaded at the top, `dss`), your targets and seeds, and an out-directory (`outdir`) which can be `none` or a path where you want to save this to memory or just return the connectome.   
A lot of this code is based on this: https://github.com/PyMVPA/PyMVPA/blob/master/mvpa2/algorithms/connectivity_hyperalignment.py

In [ ]:
def build_connectivity_matrix(dss, seed_idx, target_idx, outdir=None):
    targets_lh, targets_rh = target_idx[0], target_idx[1]
    searchlights_lh = load_searchlights(radius, 'l') #typically we use a 13 mm searchlight
    searchlights_rh = load_searchlights(radius, 'r')
    # define our connectivity metric, the dot product of samples (which on zscored data becomes
    # correlation if you normalize by nsamples.
    conn_metric = lambda x,y: np.dot(x.samples, y.samples)/x.nsamples
    connectivity_mapper = FxyMapper(conn_metric)
    # loop through each participant & save their connectivity matrix as a pymvpa dataset so we can just stick it into the 
    # searchlight hyperalignment algorithm. 
    connectomes = []
    for ds,subj in zip(dss,utils.subjects):
        if isinstance(ds,np.ndarray):
            ds = Dataset(samples=ds)
        ds.fa['node_indices'] = np.concatenate(seed_idx) # bc seed_idx is a list of two arrays (1/hemi)
        # gets mean feature measure for each searchlight on the sparse surface
        mean_features_lh = compute_mean_features(ds.samples[:,:9372],searchlights_lh[:len(targets_lh)]) 
        mean_features_rh = compute_mean_features(ds.samples[:,9372:],searchlights_rh[:len(targets_rh)])
        mean_features = np.vstack((mean_features_lh,mean_features_rh))
        print('mean features of shape: '+str(mean_features.shape)) # this will be (1175,n_timepoints)
        conn_targets = Dataset(samples=mean_features)
        conn_targets.sa['target_ids'] = np.concatenate((targets_lh,targets_rh),axis=0)
        print('getting conn vectors for subj {s}'.format(s=subj))
        connectivity_mapper.train(conn_targets)
        connectome = connectivity_mapper.forward(ds)
        print(connectome.shape)
        connectomes.append(connectome)
    print('connectomes of shape: '+str(len(connectomes))+str(connectomes[0].shape))
    if outdir:
        outstr = outdir+'/connectomes_{b}targets.npy'.format(b=len(target_ids))
        np.save(outstr, connectomes)
        print('saved at: '+outstr)
    return connectomes
    

In [ ]:
connectomes = build_connectivity_matrix(dss, seed_idx, target_idx)

## 6. Get everything in shape for hyperalignment training
Make sure our connectomes are a list of pymvpa datasets (1/subject) with labeled node indices
and are zscored  

In [ ]:
for c in connectomes:
    if isinstance(c,np.ndarray):
        c = Dataset(samples=c)
    c.fa['node_indices'] = np.concatenate(seed_idx).copy()
    c.samples = zscore(c.samples,axis=0)

In [ ]:
# build a query engine and load your surfaces
surface = utils.get_freesurfer_surface()
radius = 20 
qe = SurfaceQueryEngine(surface, radius)

## 7. Now we're ready to run searchlight hyperalignment
Let's time it and also activate the debugger so we can track its progress.  
Then, we create an instance of searchlight hyperalignment and apply it to get our   
transformation matrices

In [7]:
# First, let's make sure that we're pointing our intermediate, temporary file writing to our scratch directory.
# where to write out intermediate files
# it's arbitrary that I set all these variables; i am just lazy and do not remember what the environ variable is
os.environ['TMPDIR'] = '/dartfs-hpc/scratch/f002d44/temp'
os.environ['TEMP'] = '/dartfs-hpc/scratch/f002d44/temp'
os.environ['TMP'] = '/dartfs-hpc/scratch/f002d44/temp'

NameError: name 'os' is not defined

In [ ]:
t0 = time.time()
print('-------- beginning hyperalignment at {t0} --------'.format(t0=t0))
debug.active += ['SHPAL', 'SLC']

N_PROCS=16
N_BLOCKS=128

slhyper = SearchlightHyperalignment(queryengine=qe, # pass it our surface query engine
									nproc=N_PROCS, # the number of processes we want to use
									nblocks=N_BLOCKS, # the number of blocks we want to divide that into (the more you have the less memory it takes)
									mask_node_ids=node_indices, # tell it which nodes you are masking 
									dtype ='float64')

transformations = slhyper(cnx)
elapsed = time.time()-t0
print('-------- time elapsed: {elapsed} --------'.format(elapsed=elapsed))
h5save(outdir+'connectivity_hyperalignment_mappers.hdf5.gz', transformations, compression=9)


## 8. You did it! Way to go. 
That saved a HDF5 file of each subject's transformation matrices into the common space. 
Now we save each individual's mapper as a npz.


In [ ]:
from scipy.sparse import save_npz, load_npz

transformations = h5load(outdir+'connectivity_hyperalignment_mappers.hdf5.gz')
for T, subj in zip(transformations, utils.subjects):
	save_npz(outdir+'sub{s}_ha_mapper.npz'.format(s=subj), T._proj)
print('done saving individual mappers')

## 9. Now we are going to apply these individual mappers to our test data to validate

In [ ]:
test_data = get_data(train=False) # get our test runs
aligned_data  = []
for subj, ds in zip(budapest_subjects, test_data):
	T = load_npz(outdir+'sub{s}_ha_mapper.npz'.format(s=subj))
	print(ds.shape, ds.dtype, T.shape, T.dtype)
	aligned = np.nan_to_num(zscore((np.asmatrix(ds) * T).A, axis=0)) # apply the transformation
	np.save(outdir+'sub{s}_cnx_hyperaligned_data.npy'.format(s=subj), aligned) # or you can save left and right hemispheres separately if you so desire.
	print('done with subj {s}'.format(s=subj))
    aligned_data.append(aligned)

In [ ]:
def vertex_isc(data):
    all_results = []
    all_subjs = np.arange(data.shape[0])
    # loop through all vertices
    for v in np.arange(data.shape[2]):
        results = []
        data_v = data[:,:,v]
        for subj, ds in enumerate(data_v):
            group = np.setdiff1d(all_subjs, subj) # make groups
            group_avg = np.mean(data_v[group,:], axis=0).ravel()
            r = np.corrcoef(group_avg, ds.ravel())[0,1]
            results.append(r)
        all_results.append(np.mean(np.array(results)))
    res = np.array(all_results)
    np.save(outdir+'/vertex_isc.npy', res)
    return res

In [ ]:
isc_results = vertex_isc(aligned_data)